In [1]:
import numpy as np
import pandas as pd
import awkward_pandas as akpd
import uproot
import pyarrow.parquet as pq
import os
from pathlib import Path
import glob

In [2]:
# Where to get the dictionary key file from
dictFile = "/groups/hep/kinch/H_Zg/samples_processed/variable_Zeegamma.txt"

# Path to where our ntuples are stored
# file_path = "/groups/hep/petersen/HiggsToZgamma/AnalysisFiles2024"
# file_path = "/groups/hep/kinch/data/mc23_13p6TeV.700770.Sh_2214_eegamma.deriv.DAOD_PHYS.e8514_s4162_r14622_p6266"
file_path = "/groups/hep/kinch/data/MC/mc23_13p6TeV.601189.PhPy8EG_AZNLO_Zee.deriv.DAOD_PHYS.e8514_s4162_r15540_p6266"


# Sample file path
sampleFile_path = 'DAOD_PHYS.40055511._000408.pool.root.1'

# Path to where we save the files
save_path = '/groups/hep/kinch/H_Zg/samples_processed/MC23'

# Name of the Total DataFrame
nameTot = "Zee_deriv"

In [3]:
# Let's start by opening a single ntuple file and looking at what's inside - we'll use uproot to do so
sampleFile = uproot.open(file_path + "/" + sampleFile_path)
sampleTree = sampleFile["CollectionTree"]
sampleKeys = sampleTree.keys()

print(sampleKeys)


# MC_variableList = [8,33,76,91,127,128,135,138] #rows to skip from the variable list
data = pd.read_csv(dictFile, header = None, delim_whitespace=True)#, skiprows=MC_variableList)
print(len(data))
dataFile, dataKey = data[0], data[1]
content_DAOD= {dataKey[i]: dataFile[i] for i in np.arange(len(dataKey))}
print(content_DAOD)
print(len(content_DAOD))

['xTrigDecisionAux.', 'METAssoc_AntiKt4EMPFlowAux.', 'METAssoc_AntiKt4EMTopoAux.', 'EventInfoAux.', 'Kt4EMPFlowEventShapeAux.', 'Kt4EMPFlowNeutEventShapeAux.', 'Kt4EMPFlowPUSBEventShapeAux.', 'Kt4EMTopoOriginEventShapeAux.', 'NeutralParticleFlowIsoCentralEventShapeAux.', 'NeutralParticleFlowIsoForwardEventShapeAux.', 'TopoClusterIsoCentralEventShapeAux.', 'TopoClusterIsoForwardEventShapeAux.', 'BTagging_AntiKt4EMPFlowAux.', 'BTagging_AntiKtVR30Rmax4Rmin02TrackAux.', 'BornLeptonsAux.', 'CombinedMuonTrackParticlesAux.', 'DiTauJetsAux.', 'DiTauJetsLowPtAux.', 'ElectronsAux.', 'ExtrapolatedMuonTrackParticlesAux.', 'FourLeptonVerticesAux.', 'GSFConversionVerticesAux.', 'GSFTrackParticlesAux.', 'HLTNav_RepackedFeatures_METAux.', 'HLTNav_RepackedFeatures_ParticleAux.', 'HLTNav_Summary_DAODSlimmedAux.', 'HardScatterParticlesAux.', 'HardScatterVerticesAux.', 'InDetForwardTrackParticlesAux.', 'InDetTrackParticlesAux.', 'InTimeAntiKt4TruthJetsAux.', 'MET_Core_AntiKt4EMPFlowAux.', 'MET_Core_AntiKt

/tmp/ipykernel_2495781/4226999304.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(dictFile, header = None, delim_whitespace=True)#, skiprows=MC_variableList)


In [5]:
# We run over the different files to save a simplified version of them.
pathlist = Path(file_path).rglob('*DAOD_PHYS*.root.1')
i = 0

for path in pathlist:
    i+=1
    print(path)
    path_in_str = str(path)
    tree = uproot.open(f"{path_in_str}")#:CollectionTree")
    tree = tree["CollectionTree"]
    arrays = tree.arrays(dataKey, library="pd").rename(columns=content_DAOD)
    save_loc = f"{save_path}/{path_in_str[167:-6]}Zee_deriv{i}.parquet" #-------ændr navn her--------#
    arrays.to_parquet(path=save_loc)
    print(save_loc)
    print(f"{i} saved")

/groups/hep/kinch/data/MC/mc23_13p6TeV.601189.PhPy8EG_AZNLO_Zee.deriv.DAOD_PHYS.e8514_s4162_r15540_p6266/DAOD_PHYS.40133840._000309.pool.root.1
/groups/hep/kinch/H_Zg/samples_processed/MC23/Zee_deriv1.parquet
1 saved
/groups/hep/kinch/data/MC/mc23_13p6TeV.601189.PhPy8EG_AZNLO_Zee.deriv.DAOD_PHYS.e8514_s4162_r15540_p6266/DAOD_PHYS.40055511._000408.pool.root.1
/groups/hep/kinch/H_Zg/samples_processed/MC23/Zee_deriv2.parquet
2 saved
/groups/hep/kinch/data/MC/mc23_13p6TeV.601189.PhPy8EG_AZNLO_Zee.deriv.DAOD_PHYS.e8514_s4162_r15540_p6266/DAOD_PHYS.40133840._000111.pool.root.1
/groups/hep/kinch/H_Zg/samples_processed/MC23/Zee_deriv3.parquet
3 saved


In [6]:
#list all csv files only
csv_files = glob.glob('/groups/hep/kinch/H_Zg/samples_processed/MC23/Zee_deriv*.{}'.format('parquet'))
len(csv_files)

3

In [7]:
df_list = (pd.read_parquet(file) for file in csv_files)

# Concatenate all DataFrames
big_df   = pd.concat(df_list, ignore_index=True)
big_df

,runNumber,eventNumber,mcEventNumber,actualInteractionsPerCrossing,averageInteractionsPerCrossing,mcEventWeights,truthel_m,truthel_px,truthel_py,truthel_pz,...,GSFTrackParticlesAuxDyn_z0,GSFTrackParticlesAuxDyn_theta,GSFTrackParticlesAuxDyn_definingParametersCovMatrixDiag,GSFTrackParticlesAuxDyn_phi,GSFTrackParticlesAuxDyn_qOverP,InDetTrack_d0,InDetTrack_z0,InDetTrack_theta,InDetTrack_covdiag,InDetTrack_phi
0,410000,30230001,30230001,30.499998,30.499998,[2020.1500244140625 2020.199951171875 2001.0 2...,"[0.5109999775886536, 0.5109999775886536, 0.509...","[-12.435916900634766, -42.43939208984375, 3800...","[-67.95362854003906, -229.82901000976562, -823...","[387.378662109375, 1368.40869140625, -25520.05...",...,"[21.900915145874023, 21.939062118530273, 23.80...","[2.8746891021728516, 2.1626992225646973, 2.867...","[[0.00015933142276480794, 0.005546270869672298...","[-2.996276617050171, -0.021875210106372833, -2...","[6.995196599746123e-06, -2.56653074757196e-05,...","[-0.007209356874227524, 0.054728079587221146, ...","[21.9027099609375, 23.72264862060547, 21.94083...","[2.8746907711029053, 0.2302505075931549, 2.162...","[[0.00011652743705781177, 0.005696658045053482...","[-2.9961094856262207, -1.620420217514038, -0.0..."
1,410000,30230020,30230020,30.499998,30.499998,[2020.1500244140625 2020.199951171875 2059.100...,"[0.5099999904632568, 0.5099999904632568]","[-41985.8984375, 38364.57421875]","[6223.83154296875, -3184.657958984375]","[-118750.1171875, -26289.275390625]",...,"[-5.14871883392334, 51.58694076538086, 46.3703...","[2.744497299194336, 2.1653308868408203, 2.7983...","[[0.007766615133732557, 0.05117202550172806, 8...","[-2.933751106262207, -0.16262692213058472, 2.9...","[-0.0003075126151088625, 0.0006706993444822729...","[0.03878885507583618, -0.017793023958802223]","[46.380741119384766, 46.44047927856445]","[2.7983648777008057, 2.1706788539886475]","[[0.00019099759811069816, 0.005454188678413629...","[2.9939064979553223, -0.08253323286771774]"
2,410000,30230002,30230002,31.499998,31.499998,[2020.1500244140625 2020.199951171875 2066.300...,"[0.5099999904632568, 0.5099999904632568]","[-47873.15625, 37518.25390625]","[-4799.619140625, 21257.37109375]","[225592.453125, 191008.203125]",...,"[16.202741622924805, 16.12386131286621, 11.512...","[0.22207702696323395, 0.2100258469581604, 0.20...","[[0.00012283315300010145, 0.01041392982006073,...","[0.5150110721588135, -3.041851043701172, -3.09...","[4.432041350810323e-06, -5.054885605204618e-06...","[0.009420197457075119, 0.021715687587857246, 0...","[16.206119537353516, 16.128864288330078, 16.04...","[0.2220795899629593, 0.2100314348936081, 0.309...","[[9.318788215750828e-05, 0.010918510146439075,...","[0.5151489973068237, -3.042039394378662, -1.02..."
3,410000,30230027,30230027,30.499998,30.499998,[2020.1500244140625 2020.199951171875 2074.899...,"[0.5099999904632568, 0.5099999904632568]","[6661.857421875, -9836.8603515625]","[-24248.4453125, 19225.7109375]","[78709.3359375, -16781.3515625]",...,"[35.40622329711914, 35.342559814453125, 4.5850...","[2.231983184814453, 0.3092670142650604, 2.6160...","[[0.00018058775458484888, 0.003095927182585001...","[2.0436155796051025, -1.302966594696045, 1.230...","[3.624672535806894e-05, -1.406358842359623e-05...","[0.006965962238609791, 0.03522583469748497, -0...","[35.4052619934082, 35.337669372558594, 20.6385...","[2.2319724559783936, 0.3092612624168396, 1.290...","[[0.0001519388606538996, 0.0031701000407338142...","[2.043682336807251, -1.3039721250534058, 1.723..."
4,410000,30230039,30230039,31.499998,31.499998,[2020.1500244140625 2020.199951171875 2014.5 2...,"[0.5109999775886536, 0.5109999775886536, 0.509...","[-2.1591124534606934, -79.46436309814453, 4765...","[1.675526738166809, 53.91016387939453, -1046.4...","[-667.8734130859375, -24102.986328125, -94431....",...,"[-22.448652267456055, -22.44936180114746, 68.0...","[2.6742544174194336, 1.8772571086883545, 2.013...","[[0.00015145001816563308, 0.004154894035309553...","[-0.02179513

In [11]:
big_df.to_parquet('/groups/hep/kinch/H_Zg/samples_processed/MC23/Zee_deriv_136var.parquet')#, sep=','

In [8]:

# %%
#Laver en stor dataframe ved at sætte filerne i listen sammen
# print(f"{save_path}/{path_in_str[167:-6]}reducedMC_1marts.parquet")
# akpd.read_parquet(f"{save_path}/{path_in_str[167:-6]}reducedMC_1marts.parquet").awkward.ak.to_columns(extract_all=True)